# CRUD (CREATE, READ, UPDATE, DELETE)

A web server can be **effective and convenient gateway** for complicated databases which is why REST was created.

**Universal interface** with different programs 
- **C - Create** means you could add new items to data collection via `POST` method 
- **R - Read** means you could retrieve data stored in a collection vai `GET` method 
- **U - Update** means you could modify the contents via `PUT` method 
- **D - Delete** means you could remove

---

**R - Read**
Here we're using `json-server` with a port of *3000* that's watching our json file

```python
import requests

try:
    reply = requests.get("http://localhost:3000/cars")
except requests.RequestException:
    print("Communication error")
else:
    if reply.status_code == requests.codes.ok:
        print(reply.headers['Content-Type'])
        print(reply.json())
        print(reply.text)
    else:
        print("Server error")
```

Let's start the parsing process:
1) we create a list of properties to look up
2) create server functions to handle: *showing head*, *showing car* and to show both at the same time
3) for the show head, we use `zip(key_names, key_width)` and use `n.ljust(w)` which prints the properties with a desired length 
4) the show car function works the same way using `zip(key_names, key_widths)` except it print information for each car 
5) Then our show function prints out everything all at once 

```python

import requests

key_names = ["id", "brand", "model", "production_year", "convertible"]
key_widths = [10, 15, 10, 20, 15]


def show_head():
    for (n, w) in zip(key_names, key_widths):
        print(n.ljust(w), end='| ')
    print()


def show_car(car):
    for (n, w) in zip(key_names, key_widths):
        print(str(car[n]).ljust(w), end='| ')
    print()


def show(json):
    show_head()
    for car in json:
        show_car(car)


try:
    reply = requests.get('http://localhost:3000/cars')
except requests.RequestException:
    print('Communication error')
else:
    if reply.status_code == requests.codes.ok:
        show(reply.json())
    else:
        print('Server error')

```

Will look something like:

```
id        | brand          | model     | production_year     | convertible    | 
1         | Ford           | Mustang   | 1972                | False          |
2         | Chevrolet      | Camaro    | 1988                | True           | 
3         | Aston Martin   | Rapide    | 2010                | False          | 
4         | Maserati       | Mexico    | 1970                | False          | 
5         | Nissan         | Fairlady  | 1974                | False          | 
6         | Mercedes Benz  | 300SL     | 1957                | True           | 
```

Just remember that you could prepare a **specific** requests using `http://server:port/resource/id` where we provide an id to look at a specific car

We could also **manipulate data before sending it to the client** by using the `?` character to start additional request parameters. We could also modify that request parameter using the `&` character.
- `http://localhost:3000/cars?_sort=production_year&_order=desc`

---

So **HTTP** works like this:
1) **waits** for clients connection 
2) **reads** clients request 
3) **sends** response
4) **keeps connection alive** for the next request 
- If it's **inactive for some time** it will close the connection 

---

**D - Delete** 

We use the `delete()` function 

```python
# Similar code to the read python script but...

headers = {'Connection': 'Close'}
try:
    reply = requests.delete('http://localhost:3000/cars/1')
    print("res=" + str(reply.status_code))
    reply = requests.get('http://localhost:3000/cars/', headers=headers)
except requests.RequestException:
    print('Communication error')
else:
    print('Connection=' + reply.headers['Connection'])
    if reply.status_code == requests.codes.ok:
        show(reply.json())
    elif reply.status_code == requests.codes.not_found:
        print("Resource not found")
    else:
        print('Server error')
``` 

We see how we used `request.delete(url/id)` to remove a specific request. We also added `headers = {"Connection": "Close"}` to forcefully close the connection as opposed to keeping it open and **waiting for inactivity**. We add that to our `request.get(url, headers=headers)` request headers. 

---

**C - Create**

```python
h_close = {'Connection': 'Close'}
h_content = {'Content-Type': 'application/json'}
new_car = {'id': 7,
           'brand': 'Porsche',
           'model': '911',
           'production_year': 1963,
           'convertible': False}
print(json.dumps(new_car))
try:
    reply = requests.post('http://localhost:3000/cars', headers=h_content, data=json.dumps(new_car))
    print("reply=" + str(reply.status_code))
    reply = requests.get('http://localhost:3000/cars/', headers=h_close)
except requests.RequestException:
    print('Communication error')
else:
    print('Connection=' + reply.headers['Connection'])
    if reply.status_code == requests.codes.ok:
        show(reply.json())
    elif reply.status_code == requests.codes.not_found:
        print("Resource not found")
    else:
        print('Server error')
```

We work with the `request.post(url, headers=h_content, data=json.dumps(new_car))` which means we work with POST requests adding a header that contains a `Content-Type: application/json` and providing the argument `data` in which we dump a **json formatted** data with `json.dumps(new_car)`

---

**U - Update** 

```python
h_close = {'Connection': 'Close'}
h_content = {'Content-Type': 'application/json'}
car = {'id': 6,
       'brand': 'Mercedes Benz',
       'model': '300SL',
       'production_year': 1967,
       'convertible': True}
try:
    reply = requests.put('http://localhost:3000/cars/6', headers=h_content, data=json.dumps(car))
    print("res=" + str(reply.status_code))
    reply = requests.get('http://localhost:3000/cars/', headers=h_close)
except requests.RequestException:
    print('Communication error')
else:
    print('Connection=' + reply.headers['Connection'])
    if reply.status_code == requests.codes.ok:
        show(reply.json())
    elif reply.status_code == requests.codes.not_found:
        print("Resource not found")
    else:
        print('Server error')
```

Similar with POST, we added the `Content-Type:application/json` to the headers in our `request.put(url, headers=h_content, data=json.dumps(car))` function. Everything is similar to the POST request; **except** for the url as you need to provide a **specific id** to change a specific request.
